<a href="https://colab.research.google.com/github/gumendes/atividades-extensionistas-gumendes/blob/main/Modelo_Predi%C3%A7%C3%A3o.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# MODELO PREDITIVO - RANDOM FOREST
# Versão Simplificada e Otimizada
# ========================================

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, confusion_matrix
import plotly.express as px
import plotly.graph_objects as go
import warnings
warnings.filterwarnings('ignore')

print("="*70)
print("🌲 MODELO PREDITIVO - RANDOM FOREST")
print("="*70)

# ========================================
# 1. CARREGAR DADOS HISTÓRICOS
# ========================================

print("\n📊 Carregando dados históricos (Agosto/Setembro)...")

df_treino = pd.read_csv('../content/dados_praça_ago-set.csv', sep=';', encoding='ISO-8859-1')

# Limpar e preparar dados
df_treino.columns = df_treino.columns.str.strip()
df_treino = df_treino.rename(columns={'Hor rio': 'horario', 'Nome': 'nome'})
df_treino['nome'] = df_treino['nome'].replace({'L£cia': 'Lúcia', 'J£lia': 'Júlia', 'F tima': 'Fátima'})
df_treino['data'] = pd.to_datetime(df_treino['data'])
df_treino['presente'] = df_treino['presente'].astype(int)

print(f"✅ {len(df_treino)} registros carregados")
print(f"📅 Período: {df_treino['data'].min().strftime('%d/%m/%Y')} a {df_treino['data'].max().strftime('%d/%m/%Y')}")

# ========================================
# 2. CRIAR FEATURES (VARIÁVEIS)
# ========================================

print("\n🔧 Criando features para o modelo...")

df_treino = df_treino.sort_values(['id_aluna', 'data'])

# Encoders para variáveis categóricas
le_exercicio = LabelEncoder()
le_clima = LabelEncoder()
le_dia_semana = LabelEncoder()

df_treino['tipo_exercicio_encoded'] = le_exercicio.fit_transform(df_treino['tipo_exercicio'])
df_treino['clima_encoded'] = le_clima.fit_transform(df_treino['clima'])
df_treino['dia_semana_encoded'] = le_dia_semana.fit_transform(df_treino['dia_semana'])

# Features por aluna
features_list = []

for id_aluna in df_treino['id_aluna'].unique():
    df_aluna = df_treino[df_treino['id_aluna'] == id_aluna].copy()

    # Histórico acumulado (shift para não vazar informação do futuro)
    df_aluna['total_presencas'] = df_aluna['presente'].cumsum().shift(1, fill_value=0)
    df_aluna['total_aulas'] = range(len(df_aluna))
    df_aluna['taxa_presenca_historica'] = np.where(
        df_aluna['total_aulas'] > 0,
        df_aluna['total_presencas'] / df_aluna['total_aulas'],
        0.5  # Valor padrão para primeira aula
    )

    # Média das últimas aulas
    df_aluna['presenca_ultimas_3'] = df_aluna['presente'].rolling(3, min_periods=1).mean().shift(1, fill_value=0.5)
    df_aluna['presenca_ultimas_5'] = df_aluna['presente'].rolling(5, min_periods=1).mean().shift(1, fill_value=0.5)
    df_aluna['presenca_ultimas_7'] = df_aluna['presente'].rolling(7, min_periods=1).mean().shift(1, fill_value=0.5)

    # Sequência de presenças
    sequencia = 0
    sequencias = [0]
    for presente in df_aluna['presente'].iloc[:-1]:
        if presente == 1:
            sequencia += 1
        else:
            sequencia = 0
        sequencias.append(sequencia)
    df_aluna['sequencia_presencas'] = sequencias

    features_list.append(df_aluna)

df_features = pd.concat(features_list, ignore_index=True)

print("✅ Features criadas com sucesso!")

# ========================================
# 3. PREPARAR DADOS PARA TREINO
# ========================================

print("\n📊 Preparando conjunto de treino e validação...")

# Selecionar colunas de features
feature_cols = [
    'idade',
    'taxa_presenca_historica',
    'presenca_ultimas_3',
    'presenca_ultimas_5',
    'presenca_ultimas_7',
    'sequencia_presencas',
    'tipo_exercicio_encoded',
    'clima_encoded',
    'dia_semana_encoded'
]

X = df_features[feature_cols]
y = df_features['presente']

# Dividir em treino (80%) e validação (20%)
X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print(f"✅ Treino: {len(X_train)} amostras")
print(f"✅ Validação: {len(X_val)} amostras")
print(f"📊 Taxa de presença no treino: {y_train.mean()*100:.1f}%")

# ========================================
# 4. TREINAR RANDOM FOREST
# ========================================

print("\n🌲 Treinando Random Forest...")

modelo = RandomForestClassifier(
    n_estimators=100,        # 100 árvores na floresta
    max_depth=10,            # Profundidade máxima de cada árvore
    min_samples_split=5,     # Mínimo de amostras para dividir nó
    random_state=42,         # Reproduzibilidade
    class_weight='balanced', # Balancear classes
    n_jobs=-1                # Usar todos os processadores
)

modelo.fit(X_train, y_train)

print("✅ Modelo treinado com sucesso!")

# ========================================
# 5. AVALIAR MODELO
# ========================================

print("\n📊 Avaliando performance do modelo...")

# Previsões
y_pred = modelo.predict(X_val)
y_pred_proba = modelo.predict_proba(X_val)[:, 1]

# Métricas
acuracia = accuracy_score(y_val, y_pred)
auc_roc = roc_auc_score(y_val, y_pred_proba)

print(f"\n🎯 RESULTADOS:")
print(f"   Acurácia: {acuracia*100:.2f}%")
print(f"   AUC-ROC: {auc_roc:.3f}")

# Relatório detalhado
print(f"\n📋 Relatório de Classificação:")
print(classification_report(y_val, y_pred, target_names=['Falta', 'Presente']))

# Matriz de confusão
cm = confusion_matrix(y_val, y_pred)
print(f"\n📊 Matriz de Confusão:")
print(f"   Verdadeiros Negativos (Falta prevista corretamente): {cm[0,0]}")
print(f"   Falsos Positivos (Previu presente, mas faltou): {cm[0,1]}")
print(f"   Falsos Negativos (Previu falta, mas esteve presente): {cm[1,0]}")
print(f"   Verdadeiros Positivos (Presença prevista corretamente): {cm[1,1]}")

# ========================================
# 6. IMPORTÂNCIA DAS FEATURES
# ========================================

print("\n📊 Importância das Features...")

importancias = pd.DataFrame({
    'Feature': feature_cols,
    'Importância': modelo.feature_importances_
}).sort_values('Importância', ascending=False)

print(importancias.to_string(index=False))

# Gráfico de importância
fig_imp = px.bar(
    importancias,
    x='Importância',
    y='Feature',
    orientation='h',
    title='Importância das Features - Random Forest',
    color='Importância',
    color_continuous_scale='Viridis'
)
fig_imp.update_layout(yaxis={'categoryorder':'total ascending'}, height=500)
fig_imp.show()

# ========================================
# 7. CARREGAR CALENDÁRIO DE OUTUBRO
# ========================================

print("\n📅 Carregando calendário de Outubro...")

df_outubro = pd.read_csv('../content/dados_praça_nov.csv', sep=';', encoding='ISO-8859-1')
df_outubro.columns = df_outubro.columns.str.strip()
df_outubro = df_outubro.rename(columns={'Hor rio': 'horario'})
df_outubro = df_outubro.dropna(subset=['data'])
df_outubro['data'] = pd.to_datetime(df_outubro['data'], dayfirst=True)


print(f"✅ {len(df_outubro)} aulas programadas")
print(f"📅 De {df_outubro['data'].min().strftime('%d/%m/%Y')} até {df_outubro['data'].max().strftime('%d/%m/%Y')}")

# ========================================
# 8. GERAR PREVISÕES PARA OUTUBRO
# ========================================

print("\n🔮 Gerando previsões para Outubro...")

alunas = df_treino[['id_aluna', 'nome', 'idade']].drop_duplicates()
previsoes = []

for _, aula in df_outubro.iterrows():
    for _, aluna in alunas.iterrows():
        # Buscar histórico da aluna
        historico = df_features[
            (df_features['id_aluna'] == aluna['id_aluna']) &
            (df_features['data'] < aula['data'])
        ].sort_values('data')

        # Se tem histórico, usar última linha; senão, valores padrão
        if len(historico) > 0:
            ultima = historico.iloc[-1]
            features_dict = {
                'idade': aluna['idade'],
                'taxa_presenca_historica': ultima['taxa_presenca_historica'],
                'presenca_ultimas_3': ultima['presenca_ultimas_3'],
                'presenca_ultimas_5': ultima['presenca_ultimas_5'],
                'presenca_ultimas_7': ultima['presenca_ultimas_7'],
                'sequencia_presencas': ultima['sequencia_presencas'] if ultima['presente'] == 1 else 0,
                'tipo_exercicio_encoded': le_exercicio.transform([aula['tipo_exercicio']])[0],
                'clima_encoded': le_clima.transform([aula['clima']])[0],
                'dia_semana_encoded': le_dia_semana.transform([aula['dia_semana']])[0]
            }
        else:
            # Primeira aula - valores neutros
            features_dict = {
                'idade': aluna['idade'],
                'taxa_presenca_historica': 0.5,
                'presenca_ultimas_3': 0.5,
                'presenca_ultimas_5': 0.5,
                'presenca_ultimas_7': 0.5,
                'sequencia_presencas': 0,
                'tipo_exercicio_encoded': le_exercicio.transform([aula['tipo_exercicio']])[0],
                'clima_encoded': le_clima.transform([aula['clima']])[0],
                'dia_semana_encoded': le_dia_semana.transform([aula['dia_semana']])[0]
            }

        X_pred = pd.DataFrame([features_dict])[feature_cols]
        prob_presenca = modelo.predict_proba(X_pred)[0][1]
        prob_falta = 1 - prob_presenca

        # Classificar risco
        if prob_falta > 0.6:
            risco = 'Alto'
        elif prob_falta > 0.3:
            risco = 'Médio'
        else:
            risco = 'Baixo'

        previsoes.append({
            'data': aula['data'],
            'dia_semana': aula['dia_semana'],
            'tipo_exercicio': aula['tipo_exercicio'],
            'clima': aula['clima'],
            'id_aluna': aluna['id_aluna'],
            'nome': aluna['nome'],
            'probabilidade_presenca': round(prob_presenca * 100, 1),
            'probabilidade_falta': round(prob_falta * 100, 1),
            'previsao': 'Presente' if prob_presenca > 0.5 else 'Falta',
            'risco': risco
        })

df_previsoes = pd.DataFrame(previsoes)

print(f"✅ {len(df_previsoes)} previsões geradas!")

# ========================================
# 9. ANÁLISE DAS PREVISÕES
# ========================================

print("\n📊 Análise das Previsões...")

total = len(df_previsoes)
presencas = (df_previsoes['previsao'] == 'Presente').sum()
faltas = (df_previsoes['previsao'] == 'Falta').sum()

print(f"📊 Total de previsões: {total}")
print(f"✅ Presenças previstas: {presencas} ({presencas/total*100:.1f}%)")
print(f"❌ Faltas previstas: {faltas} ({faltas/total*100:.1f}%)")

# Distribuição de risco
print(f"\n⚠️ Distribuição de Risco:")
risco_counts = df_previsoes['risco'].value_counts()
for risco, count in risco_counts.items():
    print(f"   {risco}: {count} ({count/total*100:.1f}%)")

# Top 5 alunas em risco
print(f"\n🔴 Top 5 Alunas com Maior Risco Médio:")
risco_medio = df_previsoes.groupby('nome')['probabilidade_falta'].mean().sort_values(ascending=False).head(5)
for i, (nome, prob) in enumerate(risco_medio.items(), 1):
    print(f"   {i}. {nome}: {prob:.1f}% de probabilidade média de falta")

# Aulas com mais risco
print(f"\n📅 Aulas com Mais Alunas em Alto Risco:")
for data in df_previsoes['data'].unique():
    aula = df_previsoes[df_previsoes['data'] == data]
    alto_risco = aula[aula['risco'] == 'Alto']

    if len(alto_risco) > 0:
        tipo_ex = aula.iloc[0]['tipo_exercicio']
        clima = aula.iloc[0]['clima']
        print(f"\n   📅 {data.strftime('%d/%m/%Y')} - {tipo_ex} ({clima})")
        print(f"      🔴 {len(alto_risco)} alunas em alto risco:")

        for _, aluna in alto_risco.head(3).iterrows():
            print(f"         • {aluna['nome']}: {aluna['probabilidade_falta']:.1f}%")

# ========================================
# 10. EXPORTAR RESULTADOS
# ========================================

print("\n💾 Exportando resultados...")

# 1. Todas as previsões
df_previsoes.to_csv('previsoes_outubro_rf.csv', index=False, encoding='utf-8-sig')
print("✅ 'previsoes_outubro_rf.csv' salvo!")

# 2. Resumo por aula
resumo_aulas = df_previsoes.groupby(['data', 'tipo_exercicio', 'clima']).agg({
    'probabilidade_presenca': 'mean',
    'probabilidade_falta': 'mean',
    'id_aluna': 'count'
}).reset_index()
resumo_aulas.columns = ['Data', 'Tipo Exercício', 'Clima',
                        'Prob. Presença Média (%)', 'Prob. Falta Média (%)', 'Total Alunas']
resumo_aulas.to_csv('resumo_aulas_outubro_rf.csv', index=False, encoding='utf-8-sig')
print("✅ 'resumo_aulas_outubro_rf.csv' salvo!")

# 3. Alunas prioritárias (alto risco)
prioritarias = df_previsoes[df_previsoes['risco'] == 'Alto'].sort_values(
    ['data', 'probabilidade_falta'], ascending=[True, False]
)
prioritarias.to_csv('alunas_prioritarias_outubro_rf.csv', index=False, encoding='utf-8-sig')
print("✅ 'alunas_prioritarias_outubro_rf.csv' salvo!")

# ========================================
# 11. VISUALIZAÇÕES
# ========================================

print("\n📊 Gerando visualizações...")

# Gráfico 1: Evolução da taxa de presença prevista
fig1 = px.line(
    resumo_aulas,
    x='Data',
    y='Prob. Presença Média (%)',
    title='Previsão de Taxa de Presença - Outubro 2025',
    markers=True,
    labels={'Prob. Presença Média (%)': 'Taxa de Presença Prevista (%)'}
)
fig1.update_traces(line_color='#2ecc71', line_width=3, marker=dict(size=12))
fig1.update_layout(height=500)
fig1.show()

# Gráfico 2: Distribuição de risco por data
risco_por_data = df_previsoes.groupby(['data', 'risco']).size().reset_index(name='count')
fig2 = px.bar(
    risco_por_data,
    x='data',
    y='count',
    color='risco',
    title='Distribuição de Risco por Aula - Outubro 2025',
    labels={'count': 'Número de Alunas', 'data': 'Data', 'risco': 'Nível de Risco'},
    color_discrete_map={'Alto': '#e74c3c', 'Médio': '#f39c12', 'Baixo': '#2ecc71'}
)
fig2.update_layout(height=500)
fig2.show()

# Gráfico 3: Top 10 alunas em risco
top10_risco = df_previsoes.groupby('nome')['probabilidade_falta'].mean().sort_values(ascending=False).head(10)
fig3 = px.bar(
    x=top10_risco.values,
    y=top10_risco.index,
    orientation='h',
    title='Top 10 Alunas com Maior Risco Médio - Outubro',
    labels={'x': 'Probabilidade Média de Falta (%)', 'y': 'Aluna'},
    color=top10_risco.values,
    color_continuous_scale='Reds'
)
fig3.update_layout(height=500, yaxis={'categoryorder':'total ascending'})
fig3.show()

# Gráfico 4: Heatmap de risco por aluna e data
pivot_risco = df_previsoes.pivot(index='nome', columns='data', values='probabilidade_falta')
fig4 = px.imshow(
    pivot_risco,
    title='Heatmap de Risco de Falta - Outubro 2025',
    labels=dict(x="Data", y="Aluna", color="Prob. Falta (%)"),
    color_continuous_scale='RdYlGn_r',
    aspect="auto"
)
fig4.update_layout(height=600)
fig4.show()

# ========================================
# 12. RESUMO FINAL
# ========================================

print("\n" + "="*70)
print("✅ MODELO RANDOM FOREST - CONCLUÍDO COM SUCESSO!")
print("="*70)
print(f"\n🌲 Modelo: Random Forest (100 árvores)")
print(f"🎯 Acurácia: {acuracia*100:.2f}%")
print(f"📈 AUC-ROC: {auc_roc:.3f}")
print(f"\n📊 Previsões geradas: {len(df_previsoes)}")
print(f"⚠️ Alunas em alto risco: {len(prioritarias)}")


🌲 MODELO PREDITIVO - RANDOM FOREST

📊 Carregando dados históricos (Agosto/Setembro)...
✅ 520 registros carregados
📅 Período: 01/08/2025 a 29/09/2025

🔧 Criando features para o modelo...
✅ Features criadas com sucesso!

📊 Preparando conjunto de treino e validação...
✅ Treino: 416 amostras
✅ Validação: 104 amostras
📊 Taxa de presença no treino: 63.2%

🌲 Treinando Random Forest...
✅ Modelo treinado com sucesso!

📊 Avaliando performance do modelo...

🎯 RESULTADOS:
   Acurácia: 61.54%
   AUC-ROC: 0.507

📋 Relatório de Classificação:
              precision    recall  f1-score   support

       Falta       0.46      0.29      0.35        38
    Presente       0.66      0.80      0.73        66

    accuracy                           0.62       104
   macro avg       0.56      0.55      0.54       104
weighted avg       0.59      0.62      0.59       104


📊 Matriz de Confusão:
   Verdadeiros Negativos (Falta prevista corretamente): 11
   Falsos Positivos (Previu presente, mas faltou): 27
   


📅 Carregando calendário de Outubro...
✅ 14 aulas programadas
📅 De 01/10/2025 até 31/10/2025

🔮 Gerando previsões para Outubro...
✅ 280 previsões geradas!

📊 Análise das Previsões...
📊 Total de previsões: 280
✅ Presenças previstas: 249 (88.9%)
❌ Faltas previstas: 31 (11.1%)

⚠️ Distribuição de Risco:
   Médio: 196 (70.0%)
   Baixo: 77 (27.5%)
   Alto: 7 (2.5%)

🔴 Top 5 Alunas com Maior Risco Médio:
   1. Adelaide: 51.1% de probabilidade média de falta
   2. Ana: 46.5% de probabilidade média de falta
   3. Tania Maria: 46.2% de probabilidade média de falta
   4. Rosa: 44.6% de probabilidade média de falta
   5. Vera: 43.8% de probabilidade média de falta

📅 Aulas com Mais Alunas em Alto Risco:

   📅 22/10/2025 - Alongamento (Frio)
      🔴 2 alunas em alto risco:
         • Teresa: 66.9%
         • Vera: 63.1%

   📅 31/10/2025 - Alongamento (Frio)
      🔴 5 alunas em alto risco:
         • Ivone: 68.2%
         • Rosa: 69.7%
         • Vera: 72.6%

💾 Exportando resultados...
✅ 'previsoes


✅ MODELO RANDOM FOREST - CONCLUÍDO COM SUCESSO!

🌲 Modelo: Random Forest (100 árvores)
🎯 Acurácia: 61.54%
📈 AUC-ROC: 0.507

📊 Previsões geradas: 280
⚠️ Alunas em alto risco: 7

💾 Arquivos exportados:
   1. previsoes_outubro_rf.csv
   2. resumo_aulas_outubro_rf.csv
   3. alunas_prioritarias_outubro_rf.csv

🎯 Use o arquivo 'alunas_prioritarias_outubro_rf.csv'
   para planejar ações de incentivo!
